In [69]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated, Literal
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import operator
from pydantic import BaseModel, Field
import os

load_dotenv()

True

In [70]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.95,
    top_k=40,
    google_api_key=os.getenv("GEMINI_API_KEY")
)


In [71]:
# state
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

structured_evaluator_llm = model.with_structured_output(TweetEvaluation)

In [72]:
def generate_tweet(state: TweetState):
    # Generate a tweet based on the topic
    system_prompt = SystemMessagePromptTemplate.from_template(
        f"You are a social media expert. Generate a tweet about {state['topic']}."
    )
    human_prompt = HumanMessagePromptTemplate.from_template(
        f"{state['topic']} Rules: 1. Be concise. 2. Use hashtags. 3. Engage the audience. 4. {state['iteration']} iteration(s) done. Max iterations: {state['max_iteration']}."
    )

    messages = [system_prompt, human_prompt]

    response = model.invoke(messages)
    return {'tweet': response}

In [ ]:
def evaluate_tweet(state: TweetState):
    system_prompt = SystemMessagePromptTemplate.from_template(
        "You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."
    )
    human_prompt = HumanMessagePromptTemplate.from_template(
        f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?
2. Humor – Did it genuinely make you smile, laugh, or chuckle?
3. Punchiness – Is it short, sharp, and scroll-stopping?
4. Virality Potential – Would people retweet or share it?
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"
- feedback: One paragraph explaining the strengths and weaknesses
"""
    )

    messages = [
        system_prompt.format(),
        human_prompt.format()
    ]

    response = structured_evaluator_llm.invoke(messages)

    return {'evaluation': response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [ ]:
def optimize_tweet(state: TweetState):
    system_prompt = SystemMessagePromptTemplate.from_template(
        "You punch up tweets for virality and humor based on given feedback."
    )
    human_prompt = HumanMessagePromptTemplate.from_template(
        f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
"""
    )

    messages = [system_prompt, human_prompt]

    response = model.invoke(messages)
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [74]:
graph = StateGraph(TweetState)
graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {'approved': END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()

In [75]:
initial_state = {
    'topic': 'Artificial Intelligence',
    'iteration': 0,
    'max_iteration': 3,
}

workflow.invoke(initial_state)

NotImplementedError: Unsupported message type: <class 'langchain_core.prompts.chat.SystemMessagePromptTemplate'>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 